In [ ]:
import os
import glob
import datetime
import cftime
from collections import Counter
import calendar

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import genextreme as gev

from unseen import fileio
from unseen import time_utils
from unseen import indices
from unseen import general_utils

In [ ]:
# Parameters
dpi = 300

In [ ]:
assert os.path.isfile(metadata_file), "Must provide a metadata file (papermill option -p metadata_file [filepath])"
assert os.path.isfile(shapefile), "Must provide a shapefile (papermill option -p shapefile [filepath])"
assert os.path.isfile(nino_file), "Must provide a Nino 3.4 file (papermill option -p nino_file [filepath])"
assert 'rx15day_file' in locals(), "Must provide an rx15day output file (papermill option -p rx15day_file [filepath])"
assert 'region_name' in locals(), "Must provide a region name (papermill option -p region_name [name])"

## Spatially aggregated data

In [ ]:
agcd_files = glob.glob('/g/data/xv83/agcd-csiro/precip/daily/precip-total_AGCD-CSIRO_r005_*_daily.nc')
agcd_files.sort()

In [ ]:
ds_list = []
for infile in agcd_files:
    print(infile)
    ds = fileio.open_dataset(
        infile,
        metadata_file=metadata_file,
        shapefile=shapefile,
        variables=['pr'],
        spatial_agg='mean', 
    )
    ds = ds.compute()
    ds_list.append(ds)

In [ ]:
ds = xr.concat(ds_list, dim='time')

In [ ]:
ds

In [ ]:
ds = ds.compute()

In [ ]:
ds_monthly_totals = ds.resample({'time': 'M'}).sum('time', keep_attrs=True)
ds_monthly_totals

In [ ]:
ds_monthly_clim = ds_monthly_totals.groupby('time.month').mean('time', keep_attrs=True)
ds_monthly_clim

In [ ]:
ds_monthly_clim['pr'].plot()
plt.ylabel('Monthly average precipitation (mm)')
plt.title(f'Rainfall climatology for {region_name}')
xticks = np.arange(1,13)
xlabels = [calendar.month_abbr[i] for i in xticks]
plt.xticks(xticks, xlabels)
plt.show()

## Calculate rx15day

In [ ]:
def str_to_cftime(datestring, cftime_type):
    """Convert a date string to cftime object"""

    dt = datetime.datetime.strptime(datestring, "%Y-%m-%d")
    cfdt = cftime_type(dt.year, dt.month, dt.day)

    return cfdt


def calc_rx15day(ds):
    """Calculate rx15day values and event dates"""
    
    ds_15day = ds.rolling({'time': 15}).sum(keep_attrs=True)

    ds_rx15day = time_utils.temporal_aggregation(ds_15day, 'A-AUG', 'D', 'max', ['pr'])
    cftime_type = type(ds_rx15day['time'].values[0])

    ds_rx15day_argmax = ds_15day.resample(time='A-AUG', label='left', loffset=datetime.timedelta(days=1)).reduce(np.nanargmax, dim='time')
    time_diffs = ds_rx15day_argmax['pr'].values.astype('timedelta64[D]')
    str_times = [time.strftime("%Y-%m-%d") for time in ds_rx15day_argmax['time'].values]
    event_datetimes_np = np.array(str_times, dtype='datetime64') + time_diffs
    event_datetimes_str = np.datetime_as_string(event_datetimes_np)
    event_datetimes_cftime = [str_to_cftime(time, cftime_type) for time in event_datetimes_str]
    ds_rx15day = ds_rx15day.assign(event_time=event_datetimes_cftime)
    
    return ds_rx15day

In [ ]:
ds_rx15day = calc_rx15day(ds)

In [ ]:
#time_stamp = datetime.datetime.now().strftime("%a %b %d %H:%M:%S %Y")
#ds_rx15day.attrs['history'] = f'{time_stamp}: /home/599/dbi599/east-coast-rain/AGCD_{region_name}.ipynb (git@github.com:AusClimateService/east-coast-rain)'
#fileio.to_zarr(ds_rx15day, rx15day_file)

## Analyse and plot Rx15day data

In [ ]:
ds_rx15day['pr'].plot()
plt.title(f'Annual (Sep-Aug) Rx15day for {region_name} (AGCD)')
plt.ylabel('precipitation (mm)')
plt.xlabel('year')
plt.savefig(
    f'/g/data/xv83/dbi599/east-coast-rain/figures/Rx15day_timeseries_AGCD_{region_name}.png',
    bbox_inches='tight',
    facecolor='white',
    dpi=dpi
)
plt.show()

In [ ]:
years = ds_rx15day['time'].dt.year.values
df_rx15day = pd.DataFrame(index=years)
df_rx15day['pr'] = ds_rx15day['pr'].values

In [ ]:
df_rx15day['pr'].sort_values(ascending=False).head(n=10)

In [ ]:
rx15day_max = df_rx15day['pr'].values.max()
print(rx15day_max)

In [ ]:
event_months = ds_rx15day['event_time'].dt.month.values
month_counts = Counter(event_months)
months = np.arange(1, 13)
counts = [month_counts[month] for month in months]

plt.bar(months, counts)
plt.title(f'Rx15day timing for {region_name} (AGCD)')
plt.ylabel('number of events')
plt.xlabel('month')
xlabels = [calendar.month_abbr[i] for i in months]
plt.xticks(xticks, xlabels)
plt.savefig(
    f'/g/data/xv83/dbi599/east-coast-rain/figures/Rx15day_timing_AGCD_{region_name}.png',
    bbox_inches='tight',
    facecolor='white',
    dpi=dpi
)
plt.show()

Nino3.4 anomaly downloaded from: https://psl.noaa.gov/gcos_wgsp/Timeseries/Nino34/

In [ ]:
nino34_df = pd.read_csv(
    nino_file,
    header=None,
    skipfooter=7,
    skiprows=1,
    delim_whitespace=True,
    index_col=0,
    names=['year', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
)
nino34_df

In [ ]:
event_months = ds_rx15day['event_time'].dt.month.values
event_years = ds_rx15day['event_time'].dt.year.values
event_nino34 = []
for month, year in zip(event_months, event_years):
     event_nino34.append(nino34_df.loc[nino34_df.index == year][month].values[0])
event_nino34 = np.array(event_nino34)
event_nino34

In [ ]:
plt.scatter(event_nino34, ds_rx15day['pr'].values)
plt.title(f'Rx15day vs Nino 3.4 (AGCD)')
plt.ylabel('Rx15day (mm)')
plt.xlabel('Nino 3.4 SST anomaly (C)')
plt.axvline(0.65)
plt.axvline(-0.65)
plt.savefig(
    f'/g/data/xv83/dbi599/east-coast-rain/figures/Rx15day_ENSO_AGCD_{region_name}.png',
    bbox_inches='tight',
    facecolor='white',
    dpi=dpi
)
plt.show()

In [ ]:
def gev_analysis(ds, event, region, savefig=False):
    """Perform GEV analysis
    
    Args:
      ds (Pandas Series): Data sample
      event (float) : Event of interest
      region (str) : Name of spatial region
    """

    gev_shape, gev_loc, gev_scale = indices.fit_gev(ds.values)
    print(f'Shape parameter: {gev_shape:.2f}')
    print(f'Location parameter: {gev_loc:.2f}')
    print(f'Scale parameter: {gev_scale:.2f}')

    fig, ax = plt.subplots(figsize=[10, 8])
    gev_xvals = np.arange(0, 700)
    ds.plot.hist(bins=40, density=True, color='tab:green', alpha=0.5)
    gev_pdf = gev.pdf(gev_xvals, gev_shape, gev_loc, gev_scale)
    plt.plot(gev_xvals, gev_pdf, color='tab:green', linewidth=4.0)
    plt.xlabel('precipitation (mm)')
    plt.ylabel('probability')
    plt.title(f'Annual (Sep-Aug) Rx15day for {region} (AGCD)')
    if savefig:
        plt.savefig(
            f'/g/data/xv83/dbi599/east-coast-rain/figures/Rx15day_histogram_AGCD_{region}.png',
            bbox_inches='tight',
            facecolor='white',
            dpi=dpi
        )
    plt.show()
    
    event_probability = gev.sf(event, gev_shape, loc=gev_loc, scale=gev_scale)
    event_return_period = 1. / event_probability
    event_percentile = (1 - event_probability) * 100
    print(f'{event_return_period:.0f} year return period\n')
    print(f'{event_percentile:.2f}% percentile')

In [ ]:
gev_analysis(df_rx15day['pr'], rx15day_max, region_name, savefig=True)

In [ ]:
gev_analysis(df_rx15day['pr'][:-1], rx15day_max, region_name)